# Getting started - level 3

In this tutorial we will explore a little bit more advanced example of a program that require some configuration, requirements setup, etc. 

Again we will start with writing code for our program and saving it to [./source_files/gs_level_3.py](./source_files/gs_level_3.py) file.
This time, our program will run an estimator as a parallel function, computing the expectation value of a single observable over a set of random circuits. The results will be saved to a database, which means it will be stored in a formatted way and later on we can fetch results of or programs without looking at logs.

```python
# source_files/gs_level_3.py

from qiskit import QuantumCircuit
from qiskit.circuit.random import random_circuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator

from quantum_serverless import QuantumServerless, ditribute_task, get, put, save_result

# 1. let's annotate out function to convert it
# to function that can be executed remotely
# using `ditribute_task` decorator
@ditribute_task()
def my_function(circuit: QuantumCircuit, obs: SparsePauliOp):
    """Compute expectation value of an obs given a circuit"""
    return Estimator().run([circuit], [obs]).result().values


# 2. Next let's create our serverless object that we will be using to create context
# which will allow us to run functions in parallel
serverless = QuantumServerless()

circuits = [random_circuit(2, 2) for _ in range(3)]

# 3. create serverless context which will allow us to run functions in parallel
with serverless.context():
    # 4. The observable is the same for all expectation value calculations. So we can put that object into remote storage since it will be shared among all executions of my_function.
    obs_ref = put(SparsePauliOp(["ZZ"]))

    # 5. we can run our function for a single input circuit 
    # and get back a reference to it as now our function is a remote one
    function_reference = my_function(circuits[0], obs_ref)

    # 5.1 or we can run N of them in parallel (for all circuits)
    # note: if we will be using real backends (QPUs) we should either use
    #       N separate backends to run them in parallel or
    #       one will be running after each other sequentially
    function_references = [my_function(circ, obs_ref) for circ in circuits]

    # 6. to get results back from reference
    # we need to call `get` on function reference
    single_result = get(function_reference)
    parallel_result = get(function_references)
    print("Single execution:", single_result)
    print("N parallel executions:", parallel_result)

    # 6.1 (Optional) write results to db.
    save_result({
        "status": "ok",
        "single": single_result.tolist(),
        "parallel_result": [entry.tolist() for entry in parallel_result]
    })

```

As you can see we move to advanced section of using serverless. 

Here we are using `ditribute_task` decorator to convert our function to asynchronous distributed one. 
With that `my_function` is converted into asynchronous distributed function (as a result you will be getting function pointer), which means that the function no longer executes as part of your local python process, but executed on configured compute resources.

Moreover, we are using `save_result` function in order to save results into database storage, so we can retrieve it later after program execution.

Next we need to run this program. For that we need to import necessary modules and configure QuantumServerless client. We are doing so by providing name and host for deployed infrastructure.

In [16]:
from quantum_serverless import QuantumServerless, GatewayProvider

In [17]:
provider = GatewayProvider(
    username="user", # this username has already been defined in local docker setup and does not need to be changed
    password="password123", # this password has already been defined in local docker setup and does not need to be changed
    host="http://gateway:8000", # address of provider
)

serverless = QuantumServerless(provider)
serverless

<QuantumServerless | providers [gateway-provider]>

Run program

In [18]:
from quantum_serverless import Program 

program = Program(
    title="Advanced program",
    entrypoint="gs_level_3.py",
    working_dir="./source_files/"
)

job = serverless.run_program(program)
job

<Job | 65dadf22-16d5-40e2-9a3e-de460e439c34>

In [20]:
job.status()

'SUCCEEDED'

With `job.result()` we can get saved results inside of our function back. `.result()` call will return you whatever you passed in `save_result` inside the program file, while `.logs()` will return everything that was logged by job (stdio, e.g prints).

In [21]:
job.result()

'{"status": "ok", "single": [1.0], "parallel_result": [[1.0], [1.0], [1.0]]}'